In [1]:
import numpy as np
import pandas as pd 
from opencage.geocoder import OpenCageGeocode
import geopy.distance

key = '0573d875005f45bebe452ff81c26b684'
geocoder = OpenCageGeocode(key)

class desafioBiodiversidade():
    def __init__(self,filePath):
        try:
            f = open(filePath,"r")
            self.originalData = f.readlines()
            self.data = [l.rstrip().split(";") for l in self.originalData]
            self.len = len(self.data)-1
            f.close()
        except:
            print("Error: cannot open file.")
            data = None  

    # Tópico 1 - Dados Faltantes
    def retornaMediaDadosFaltantesPorColuna(self,n):
        data = self.data
        
        nblank = [0]*len(data[0])
  
        for line in data:
            for ind,column in enumerate(line):
                if column == "Sem Informações" or column == "":
                    nblank[ind]+=1
 
        mean = [np.round((x/self.len)*100,2) for x in nblank]

        dict = {'Coluna': data[0], n+'-Faltantes':nblank, n+'-Média': mean}
        df = pd.DataFrame(dict)
        return df
    
    # Tópico 2 - Nível Taxonômico
    def retornaNivelTaxonomicoCadaOcorrencia(self):
        lista = self.data

        filo = []
        for i in range(len(lista[0])):
            
            if lista[0][i] == 'Especie':
              
                for k in range(1,len(lista)):
                    
                    if lista[k][i] != 'Sem Informações':
                        filo.append('{} - Especie'.format(k))
                    
                    elif lista[k][i-1] != 'Sem Informações':
                        filo.append('{} - Genero'.format(k))
                        
                    elif lista[k][i-2] != 'Sem Informações':
                        filo.append('{} - Familia'.format(k))
                        
                    elif lista[k][i-3] != 'Sem Informações':
                        filo.append('{} - Ordem'.format(k))
                    
                    elif lista[k][i-4] != 'Sem Informações':
                        filo.append('{} - Classe'.format(k))
                        
                    elif lista[k][i-5] != 'Sem Informações':
                        filo.append('{} - Filo'.format(k))
                        
                    else:
                        filo.append('{} - Reino'.format(k))
     
        return filo 

    # Tópico 3 - Filtro de Ocorrências
    def retonaSeOcorrenciaExiste(self, filterAmeaca):
        self.AmeacaList = []
        self.dataAux = []
        self.dataT = []
        
        # Matriz transposta
        for j in range(len(self.data[0])):        
            for i in range(len(self.data)-1):
                self.dataAux.append(self.data[i][j])
            self.dataT.append(self.dataAux)
            self.dataAux = []
        
        
        # Filtro estado de conservacao
        for i in range(len(self.dataT)):
            if self.dataT[i][0] == "Estado de conservacao":
                for j in range(len(self.dataT[i])):
                    if self.dataT[i][j] == filterAmeaca :
                        self.AmeacaList.append(["Amostra {}: {}: ".format(j, self.dataT[i-12][j]), filterAmeaca])
                    
        return self.AmeacaList
    
class GeoCode:

    def __init__(self,data):
        self.data = data
        self.topics = ["country","state","state_code","city"]

    def check_localization(self):
        indlat = self.data[0].index("Latitude")	
        indpais = self.data[0].index("Pais")
        
        print(len(self.data))
        result = []
        for ind,line in enumerate(self.data[123:501]):
            
            lat = self.parse_float(line[indlat])
            lon = self.parse_float(line[indlat+1])

            geo = geocoder.reverse_geocode(lat,lon)   # retorna info de lat,lon
            comp = geo[0]['components']               # separa info de localizacao
            info = self.get_info(comp) 

            res = self.info_compare(line[indpais:indpais+3],info)
    
            if not res:
                rlat,rlon = self.get_latlon(line[indpais:indpais+3])
                dist = self.get_distance((lat,lon),(rlat,rlon))
                print(ind," ",dist)
                result.append(dist)
            else:
                result.append(True)
                print(ind, " ",True)
                
        return result



    # tenta a leitura de numeros float para latitude e longitude 
    def parse_float(self,info):
        try:
            value = float(info)
        except:
            value = 0.0
        return value


    # separa as informacoes de país, estado, código de estado e cidade
    def get_info(self,components):
        aux = []
        for elem in self.topics:
            try:
                value = components[elem]
            except:
                value = "Sem Informações"
            aux.append(value)
        return [aux[0],(aux[1],aux[2]),aux[3]]


    # compara as informacoes existentes
    def info_compare(self,line,info):

        correct = True
        for i, elem in enumerate(line):
            if line[i]!="Sem Informações" and info[i]!= "Sem Informações":
                if i==1:
                    if (line[i]!=info[i][0] and line[i]!=info[i][1]):
                        correct = False
                elif line[i]!=info[i]:
                    correct = False
        return correct 


    # busca a latitude e longitude de um endereco
    def get_latlon(self,line):

        address = self.concat_info(line) 
        geo = geocoder.geocode(address)
        lat,lon = geo[0]['geometry']['lat'], geo[0]['geometry']['lng']
        return lat,lon


    # concatena info em string para fazer a busca no geocode
    def concat_info(self,line):
        aux = ""
        for elem in line:
            if elem != "Sem Informações":
                aux += elem + ","
        return aux[:-1]


    # calcula distancia entre duas coordenadas
    def get_distance(self,coord1, coord2):
        return geopy.distance.geodesic(coord1,coord2).km
        
        
filePath1 = "portalbio_export_16-10-2019-14-39-54.csv"
filePath2 = "portalbio_export_17-10-2019-13-06-22.csv"
filePath3 = "portalbio_export_17-10-2019-13-15-00.csv"   
b1 = desafioBiodiversidade(filePath1)
b2 = desafioBiodiversidade(filePath2)
b3 = desafioBiodiversidade(filePath3)

# Análise Dados Faltantes

In [9]:
d1 = b1.retornaMediaDadosFaltantesPorColuna("1")

d2 = b2.retornaMediaDadosFaltantesPorColuna("2")
d1 = d1.join(d2.set_index('Coluna'), on='Coluna')
    
d3 = b3.retornaMediaDadosFaltantesPorColuna("3")    
d1 = d1.join(d3.set_index('Coluna'), on='Coluna')    

### Coluna com mais dados faltantes

In [10]:
print(d1[["Coluna","1-Média","2-Média","3-Média"]].to_string(index=False))

                              Coluna  1-Média  2-Média  3-Média
                 Nome da instituicao   100.00   100.00   100.00
                Sigla da instituicao     0.00     0.00     0.00
               Nome da base de dados     0.00     0.00     0.00
              Sigla da base de dados     0.00     0.00     0.00
           Responsavel pelo registro     0.08     0.00     0.01
        Numero do registro no portal     0.00     0.00     0.00
 Numero do registro na base de dados     6.01    19.28    27.71
                    Data do registro     3.50    19.28    26.17
                      Data do evento    54.45    12.25    37.26
                    Data de Carencia     0.00     0.00     0.00
                     Nome cientifico     1.69     1.80     4.87
                          Nome comum    74.21    40.54    68.44
    Nome cientifico na base de dados     0.00     0.00     0.00
                    Nivel taxonomico     1.69     1.80     4.87
                Numero de individuos    

In [11]:
print("Colunas com mais de 90% dos dados ausentes")
print( d1.loc[(d1["1-Média"]>80)&(d1["2-Média"]>80)&(d1["3-Média"]>80),"Coluna"].to_string(index=False)  )

Colunas com mais de 90% dos dados ausentes
 Nome da instituicao
               Reino
 Categoria de Ameaca
          Jurisdicao


### Média de dados faltantes por coluna nas três tabelas

In [4]:
tmean = d1[["Coluna","1-Média","2-Média","3-Média"]].mean(axis=1)
d1["Média Total"] = tmean
print(d1[["Coluna","1-Média","2-Média","3-Média","Média Total"]].to_string(index=False))

                              Coluna  1-Média  2-Média  3-Média  Média Total
                 Nome da instituicao   100.00   100.00   100.00   100.000000
                Sigla da instituicao     0.00     0.00     0.00     0.000000
               Nome da base de dados     0.00     0.00     0.00     0.000000
              Sigla da base de dados     0.00     0.00     0.00     0.000000
           Responsavel pelo registro     0.08     0.00     0.01     0.030000
        Numero do registro no portal     0.00     0.00     0.00     0.000000
 Numero do registro na base de dados     6.01    19.28    27.71    17.666667
                    Data do registro     3.50    19.28    26.17    16.316667
                      Data do evento    54.45    12.25    37.26    34.653333
                    Data de Carencia     0.00     0.00     0.00     0.000000
                     Nome cientifico     1.69     1.80     4.87     2.786667
                          Nome comum    74.21    40.54    68.44    61.063333

### Total de dados faltantes por tabela

In [25]:
print(d1[["Coluna","1-Faltantes","2-Faltantes","3-Faltantes"]].to_string(index=False))

                              Coluna  1-Faltantes  2-Faltantes  3-Faltantes
                 Nome da instituicao         3912          555        58411
                Sigla da instituicao            0            0            0
               Nome da base de dados            0            0            0
              Sigla da base de dados            0            0            0
           Responsavel pelo registro            3            0            6
        Numero do registro no portal            0            0            0
 Numero do registro na base de dados          235          107        16183
                    Data do registro          137          107        15287
                      Data do evento         2130           68        21764
                    Data de Carencia            0            0            0
                     Nome cientifico           66           10         2843
                          Nome comum         2903          225        39975
    Nome cie

In [26]:
tsum = d1[["Coluna","1-Faltantes","2-Faltantes","3-Faltantes"]].sum(axis=0)
print(tsum)

Coluna         Nome da instituicaoSigla da instituicaoNome da...
1-Faltantes                                                31956
2-Faltantes                                                 3295
3-Faltantes                                               442128
dtype: object


# Análise Nível Taxonômico

In [27]:
nt1 = b1.retornaNivelTaxonomicoCadaOcorrencia()
nt2 = b2.retornaNivelTaxonomicoCadaOcorrencia()
nt3 = b3.retornaNivelTaxonomicoCadaOcorrencia()

### Nível taxonômico por tabela 

In [29]:
level = ["Reino","Filo","Classe","Ordem","Familia","Genero","Especie"]

# Retorna lista de contadores dos níveis taxonômicos
def count_per_level(nt):    
    c = [0]*7
    for line in nt:
        c[level.index(line.split("- ")[1])] += 1
    return c

In [30]:
c1 =count_per_level(nt1)
c2 =count_per_level(nt2)
c3 =count_per_level(nt3)

# Calcula a média
mc1 = [np.round((x/b1.len)*100,2) for x in c1]
mc2 = [np.round((x/b2.len)*100,2) for x in c2]
mc3 = [np.round((x/b3.len)*100,2) for x in c3]

# Converte em dicionário e depois Dataframe 
dict = {'Nível':level, "Tabela1":c1, "Média1(%)":mc1 ,"Tabela2":c2, "Média2(%)":mc2, "Tabela3":c3, "Média3(%)":mc3}
nt = pd.DataFrame(dict)

### Porcentagem de níveis por tabela

In [9]:
print(nt[["Nível","Média1(%)","Média2(%)","Média3(%)"]].to_string(index=False))

   Nível  Média1(%)  Média2(%)  Média3(%)
   Reino       3.17       2.52       5.34
    Filo       0.59       0.36       0.56
  Classe       0.54       4.14       0.74
   Ordem       0.43       1.26       1.45
 Familia      11.78       3.42       7.16
  Genero      20.94       3.06       9.64
 Especie      62.55      85.23      75.11


# Análise filtro de ocorrências - Estado de conservação

In [31]:
estado = ["Espécie Ameaçada", "Espécie não Ameaçada"]

Aec1 = b1.retonaSeOcorrenciaExiste(estado[0])
Nec1 = b1.retonaSeOcorrenciaExiste(estado[1])

Aec2 = b2.retonaSeOcorrenciaExiste(estado[0])
Nec2 = b2.retonaSeOcorrenciaExiste(estado[1])

Aec3 = b3.retonaSeOcorrenciaExiste(estado[0])
Nec3 = b3.retonaSeOcorrenciaExiste(estado[1])

In [32]:
# Converte em dicionário e depois Dataframe 
dict = {'Estado de Conservação':estado, "Tabela1":[len(Aec1),len(Nec1)], "Média1(%)":[np.round((len(Aec1)/b1.len)*100,2),np.round((len(Nec1)/b1.len)*100,2)],\
                                        "Tabela2":[len(Aec2),len(Nec2)], "Média2(%)":[np.round((len(Aec2)/b2.len)*100,2),np.round((len(Nec2)/b2.len)*100,2)],\
                                        "Tabela3":[len(Aec3),len(Nec3)], "Média3(%)":[np.round((len(Aec3)/b3.len)*100,2),np.round((len(Nec3)/b3.len)*100,2)]}
ec = pd.DataFrame(dict)

### Número total de ocorrências da coluna "Estado de conservação"

In [33]:
print(ec[["Estado de Conservação","Tabela1","Tabela2","Tabela3"]].to_string(index=False))

Estado de Conservação  Tabela1  Tabela2  Tabela3
     Espécie Ameaçada      570       18     6203
 Espécie não Ameaçada     3329      536    52200


### Porcentagem de ocorrências

In [34]:
print(ec[["Estado de Conservação","Média1(%)","Média2(%)","Média3(%)"]].to_string(index=False))

Estado de Conservação  Média1(%)  Média2(%)  Média3(%)
     Espécie Ameaçada      14.57       3.24      10.62
 Espécie não Ameaçada      85.10      96.58      89.37


# Análise da Localização

In [2]:
gCode = GeoCode(b3.data)
result = gCode.check_localization()
print(result)

58412
0   3992.3794343011778
1   True
2   True
3   True
4   4175.08264323352
5   True
6   True
7   True
8   True
9   True
10   3981.1475283670097
11   3981.1475283670097
12   True
13   True
14   True
15   True
16   True
17   True
18   True
19   4093.915729185928
20   True
21   True
22   True
23   True
24   True
25   True
26   True
27   True
28   True
29   4093.915729185928
30   True
31   True
32   True
33   True
34   True
35   True
36   True
37   True
38   True
39   True
40   True
41   True
42   True
43   True
44   True
45   True
46   True
47   True
48   True
49   True
50   True
51   True
52   True
53   True
54   True
55   True
56   True
57   True
58   True
59   True
60   True
61   True
62   True
63   True
64   True
65   True
66   4163.42537054753
67   True
68   True
69   True
70   True
71   True
72   True
73   True
74   True
75   True
76   True
77   True
78   True
79   True
80   True
81   True
82   True
83   True
84   True
85   True
86   True
87   True
88   True
89   True
90   True
91

In [20]:
f1 = [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 7528.184504818057, 7528.184504818057, 7528.184504818057, 7528.184504818057, 7528.184504818057, True, True, True, 4149.794591208655, 4642.761948790867, 4696.053190309778, 4451.040888422669, 4644.044396518796, 5320.693830148226, 4696.204237400946, 4824.030877095455, 4013.2750853928637, 3980.1113424248174, 4641.328225993282, 4735.807777187057, 4633.232482728352, 4617.864333577603, 4633.351706509263, 4611.810519154261, 1418.8025882484749, 4569.291473221052, 1288.2823287234362, 4632.978758491025, 4574.392215576296, 1963.2906622039368, 3958.9746898387334, 3964.0031908971087, 5503.56200303842, 4940.598942191854, 2672.6033451321496, 4552.176395502553, 3418.5159159456744, 2637.49260921536, 3088.542570763768, 3409.578475758746, 2148.6583148863165, 1194.7885958497695, 4092.2344976156937, 2637.075288751355, 3958.9746898387334, 3882.765754762278, 3883.021425513163, 3883.021425513163, 5949.041422154225, 5949.010945359882, 5949.010945359882, 5949.010945359882, 5949.010945359882, 5949.010945359882, 852.6141574340858, 2070.636631871548, 2215.7101119556924, 3344.9150264345153, 4024.6940527241313, 2084.2795873805594, 1076.9214074774673, 1076.9214074774673, 3204.7788293182316, 1407.6405706241233, 3416.7074151098336, 3250.822318155013, 3312.485680878226, 3255.295242495348, 527.7069665217947, 3316.6210590855712, 2075.479023058274, 4087.286075656078, 4087.286075656078, 4561.779622305886, 3837.906872712129, 2318.601115919411, 5166.4952273812205, 2919.224765547644, 5133.072437696583, 2905.8036994149816, 3815.5466851307833, 5057.278416610584, 5057.001013773499, 5057.278416610584, 5131.246673342448, 4645.156101860414, 4632.978758491025, 3315.8945917515853, 3332.175942818518, 5154.864585228153, 4634.505018826534, True, True, 4784.546220439265, 4774.376736086217, 5166.20469062961, 3929.526545593212, 4505.131066124366, 3337.3637184255845, 5069.986434130129, 3779.512067232659, 3918.33632107565, 3932.2854331922704, 5280.82494448666, 4641.587686338287, 5358.551914950519, 4260.523610049865, 4787.653368877552, 4874.434135059219, 3802.50084687457, 4697.1469109313975, 3963.0879758625556, 3957.773529803449, 3899.4535004645054, 4669.87476701987, 4518.287258390887, 3897.8262510985587, 4076.4324061996117, 4204.341421835098, 4123.06697602132, 4771.581610537657, 4879.114343511884, 3851.308234464808, 3852.8715552625936, 4696.082963640639, 4696.082963640639, 4003.344520257193, 3981.4532347735953, 3976.033964489521, 4151.388847634416, 3994.6599949399274, 4098.811239250251, 4519.754007565725, 4511.912218189218, 1087.1879895970574, 4820.188561949253, 3420.4142368506823, 4192.3635068115855, 3362.014081628448, 1898.0668794277944, 4598.5573545946045, 4113.6960513778995, 3248.083329018699, 4360.5699056108015, 3762.7228385121657, 3967.858727916272, 3998.5010181503876, 4606.375686696269, 4706.482843643926, 3323.060499172123, 4771.581610537657, 4484.956362429499, 3957.963746111884, 4501.406364994687, 3916.7582002984605, 4633.500738872241, 3241.70129395357, 3241.70129395357, 2648.197857560985, 3298.1342984088233, 4641.328225993282, 4641.328225993282, 4644.8997277472245, 4772.04645807654, 4638.778831822186, 5080.662936159796, 4632.978758491025, 3405.9039363274223, True, True, 3935.906700531607, 4642.22498816611, 4632.897837532692, 4632.778052801025, 4132.010288889356, 4192.3635068115855, 4665.172295322156, 4065.3352142372696, 4550.215185720787, 2516.9551403744726, 4633.232482728352, 2933.742657224885, 2516.9279772261575, 4342.638224913261, 3132.2705807789425, 4786.378244254794, 4807.750774208584, 3712.8075502135716, 2162.588666639448, 3332.175942818518, 3309.341405537157, 4638.778831822186, 4697.1469109313975, 2261.670980282764, 4641.328225993282, 4665.172295322156, 4665.36961844159, 4393.634561779194, 4001.3520357028237, 3322.4509438261593, 4073.9949847472913, 972.8796126213095, 4192.3635068115855, 5166.4952273812205, 5130.737734346452, 3262.1615295697948, 2075.479023058274, 5129.25071341991, 643.2582289432535, 3927.184590558073, 643.2582289432535, 5426.397298449803, 3241.2221940698237, 3422.198077560144, 3249.1816606112616, 4633.0508580308315, 4633.0508580308315, 3344.9150264345153, 3264.2478246404585, 4160.8736422834, 4559.244173757153, 5396.64760291883, 6396.890816889809, 3928.744900221692, 4641.328225993282, 4645.551274707988, 4695.723038780772, 4222.532805542789, 3186.957316396371, 3308.728599654621, 4631.193281559976, 4638.74901110158, 4643.969861183282, 4116.7226252387545, 3739.7958679564877, 3293.8065715245857, 3332.442889235686, 4113.493039944925, 4000.665473003936, 4159.8428644543155, 5197.482331759733, 2651.001836470795, 4167.070507684009, 558.4159956859199, 3849.3657168542463, 923.8675597295115, 923.8675597295115, 5120.207929376692, 5129.045213930271, 3826.9855493212413, 4548.731751429071, 4633.232482728352, 3204.093045288388, 5131.726349339642, 4013.705619140371, 4073.9949847472913, 4789.136784754514, 1427.0812957099872, 3960.679281912966, 3773.224056669227, 4412.916492349338, 4640.066597856787, 4665.36961844159, 5078.249452173864, 4764.006053948109, 3323.060499172123, 4665.36961844159, 4665.172295322156, 4051.6426972526315, 3481.9733765170317, 4641.328225993282, 4637.22807329117, 3960.679281912966, 4765.566226792459, 4633.210001788761, 4631.511754990908, 3923.978738830059, 4820.9821863931165, 4696.053190309778, 3481.2626323420395, 4631.994386802048, 4638.778831822186, 4664.975557229778, 4641.513121241485, 5470.525934198375, 4655.219180015323, 3932.2854331922704, 3515.6754968630557, 3099.9843802012742, 804.9516990054102, 3377.4082200211897, 3243.014049829576, 3348.115313891395, 4163.426635592827, 3314.7576713203193, 4385.254978997742, 3250.02233532353, 3295.879122919191, 3390.5920317191203, 4629.607419179165, 3636.539832927423, 3916.7582002984605, 4515.372671585189, 4590.515933386583, 4528.397647249328, 3453.391002161359, 4407.059764751032, 3958.9746898387334, 5124.467623356661, 4122.56330690058, 3969.9968560340894, 4116.7226252387545, 3316.6210590855712, 4166.619330378496, 3316.6210590855712, 3315.8945917515853, 4639.185835549844, 4638.778831822186, 4699.07586147684, 4644.929564563906, 4639.185835549844, 1656.87419838488, 3761.403067836701, True, 3960.679281912966, 2064.6238274546417, 5987.979812466802, 3241.8234500995186, 3241.8234500995186, 3241.9465583504407, 3212.29291552596, 3238.9910696146344, 3960.5940562100723, 4200.692300671816, 4209.660925120763, 4030.1973366755155, 7302.034478076319, 3238.9910696146344, 4032.5204759337335, 4559.988063614927, 4633.2622913591895, 4696.082963640639, 4697.1469109313975, 4633.2622913591895, 4641.328225993282, 6247.377685245382, 3298.1342984088233, 4360.451862272557, 4451.040888422669, 4699.07586147684, 4792.911790641644, 5129.045213930271, 3960.679281912966, 4073.9949847472913, 4632.978758491025, 4812.217478153939, 4149.794591208655, 5426.397298449803, 5426.397298449803, 3212.433807339303, 3212.433807339303, 3032.105327693386, 4638.927924660483, 2392.5878038097985, 2675.0972359911943, 3969.3327063179768, 3960.5940562100723, 3969.302166883277, 3059.967952645739, 2434.7424982709767, 4559.480931901579, 6375.019897919905, 3976.033964489521, 4818.079696624468, 3913.1240393736107, 3209.1549743108435, 4071.676439660289, 5033.404635304881, 2273.826418932208, 3967.408181927255, 4149.794591208655, 3969.302166883277, 4696.142531738844, 4640.066597856787, 4665.458840421442, 4192.303197437487, 4116.7226252387545, 3054.2789769357987, 3928.767801896687, 4595.776600728798, 1075.6454182338255, 4641.328225993282, 4696.082963640639, 3960.5940562100723, 3964.088415246669, 2075.479023058274, 5761.110226632664, 3960.5940562100723, 787.342385696257, True, 6223.123312628857, 4201.772122515594, 1974.2175156271085, 3499.1243074618856, True, 3960.5940562100723, 4606.876880607659, 4662.945732204553, 3492.1512832775857, 4559.244173757153, 4661.971659076991, 2088.332083205439, 655.7312960204978, 4663.765314004606, 4608.969775849708, 4559.244173757153, 4608.969775849708, 4299.0274772524635, 4696.9631704520025, 3267.4155129338037, 4551.673825212386, 4645.719903265751, 4630.803333793618, 2902.4497845890705, 4573.356871677936, 4687.171231868817, 4015.414812810549, 3072.6448911870298, 4015.414812810549, 3243.014049829576, 3855.552018540886, 4632.978758491025, 3172.0798494874384, 4665.172295322156, 4167.070507684009, 4633.210001788761, 4027.9511626642725, 3960.679281912966, 3311.9079849864847, 3960.679281912966, 3964.088415246669, 3960.679281912966, 3960.679281912966, 3960.679281912966, 3960.679281912966, 3960.679281912966, 3960.679281912966, 3960.679281912966, 4064.6607515054056, 4038.0280037063458, True, 3184.2795870676914, 923.8675597295115, 3979.604116478029, 4809.685833153157, 4633.0508580308315, 4663.772515518338, 3733.098452122731, 4641.030126742114, 4642.761948790867, 5166.4952273812205, 5308.455477798685, 4029.3424441053985, 4643.748148227832, 3608.147754801836, 1047.0735797717575, 4632.778052801025, 5158.011696465441, 2982.837452182013, 4645.156101860414, 4530.416370377595, 4665.36961844159, 3322.4509438261593, 4632.978758491025, 757.4827728272943, 3355.5480813877994, 456.55105557035716, 1141.0355256778664]

In [21]:
f2 = [17.223145947903785, 135.08993494067386, 9.727415813407475, 12.390293311974654, 17.154772262120158, 15.481375493296326, 15.481375493296326, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 11.825349922864131, 18.017624014529066, 2.3840899911074085, 6.030109922978084, 14.77478088354493, 14.77478088354493, 14.77478088354493, 14.77478088354493, 14.77478088354493, 14.77478088354493, 14.77478088354493, 14.77478088354493, 18.343326369885318, 18.343326369885318, 18.343326369885318, 18.343326369885318, 18.343326369885318, 18.343326369885318, 18.343326369885318, 18.343326369885318, 18.343326369885318, 18.343326369885318, 18.343326369885318, 18.343326369885318, 8.850347614687674, 8.850347614687674, 8.850543480088888, 8.851620746846768, 8.850543480088888, 8.847899330497281, 5.9057858466318764, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 9.761555179042052, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 16.379909193388574, 8.850347614687674, 8.850543480088888, 8.850543480088888, 8.850543480088888, 8.850543480088888, 8.850347614687674, 8.850347614687674, 8.850347614687674, 8.850347614687674, 8.850347614687674, 8.850347614687674, 11.058932998363515, 17.27012002204468, 17.159419889583777, 8.850347614687674, 8.850347614687674, 8.850347614687674, 8.850347614687674, 8.850347614687674, 8.850347614687674, 8.850543480088888, 8.850543480088888, 8.850543480088888, 8.850347614687674, 19.732351143234222, 11.781408872995325, 11.781408872995325, 11.781261724113998, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 8.851620746846768, 8.850543480088888, 8.850543480088888, 8.850543480088888, 2.1534350780931746, 2.1534350780931746, 2.1534350780931746, 2.1534350780931746, 2.1534350780931746, 2.1534350780931746, 15.023860534342601, 8.850543480088888, 8.850543480088888, 8.850543480088888, 8.850543480088888, 6.399771986202405, 8.446419605132267, 3.595394080451636, 4.744247982038939, 4.744247982038939, 4.744247982038939, 18.067065069850305, 18.067065069850305, 18.067065069850305, 18.067065069850305, 9.788863333290598, 19.97652615698271, 17.159419889583777, 3.812102560920819, 3.812102560920819, 3.812102560920819, 3.812102560920819, 3.812102560920819, 3.812102560920819, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 11.781408872995325, 3.812102560920819, 8.06997773499552, 8.06997773499552, 8.070007210303832, 4.896018867264038, 4.896018867264038, 4.896018867264038, 4.896018867264038, 3.812102560920819, 3.812102560920819, 3.812102560920819, 11.78104100400289, 11.78104100400289, 11.78104100400289, 2.740087961414827, 0.24318083542470367, 0.24318083542470367, 8.472945840636877, 4.184881903460614, 9.764169981220256, 3.584276244724243, 3.584276244724243, 36.23658387939534, 36.23658387939534, 36.23658387939534, 36.23658387939534, 36.23658387939534, 5.119590730715554, 5.119590730715554, 6.567742046905153, 0.3484117004050689, 0.3484117004050689, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 17.933105018193544, 3.7367183800127814, 3.7367183800127814, 3.7367183800127814, 3.7367183800127814, 5.236250175927391, 5.236250175927391, 5.236250175927391, 5.7389461392422065, 5.7389461392422065, 4.286226568464779, 13.063595444705648, 13.063595444705648, 13.063595444705648, 13.063595444705648, 3.7367183800127814, 3.7367183800127814, 5.236250175927391, 5.236250175927391, 5.236250175927391, 5.236250175927391, 5.7389461392422065, 4.286226568464779, 13.063595444705648, 13.063595444705648, 13.063595444705648, 13.063595444705648, 13.063595444705648, 13.063595444705648, 18.067065069850305, 17.877466824050657, 17.877466824050657, 17.877466824050657, 17.877466824050657, 17.877466824050657, 17.877466824050657, 8.234896410569755, 8.234896410569755, 8.234896410569755, 8.234896410569755, 8.234896410569755, 8.234896410569755, 8.234896410569755, 8.234896410569755, 2.596563352313917, 2.596563352313917, 2.596563352313917, 2.596563352313917, 2.596563352313917, 18.066953617171432, 18.066953617171432, 18.067065069850305, 18.41502267487529, 18.414820686269984, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 4.542996416657622, 0.6292286383880356, 36.23658387939534, 36.23658387939534, 36.23658387939534, 36.23658387939534, 18.414517703466217, 18.414820686269984, 18.41502267487529, 18.41502267487529, 18.41502267487529, 18.41502267487529, 18.41502267487529, 18.41502267487529, 18.41502267487529, 18.41502267487529, 18.41502267487529, 12.93297818253063, 12.93297818253063, 2.3818798294938417, 2.740087961414827, 6.8269644880653075, 6.8269644880653075, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 12.93297818253063, 36.23655026637394, 36.23655026637394, 36.23655026637394, 36.23655026637394, 2.1534350780931746, 0.3532301184451777, 0.3532301184451777, 0.3532301184451777, 12.10482000671949, 11.503614912327908, 2.740087961414827, 0.26531221467635124, 19.732351143234222, 19.732351143234222, 19.732351143234222, 19.732351143234222, 0.3532301184451777, 18.067065069850305, 18.067065069850305, 7.208472922285388, 5.473328441813793, 13.434264218637333, 12.938983430203514, 6.030109922978084, 5.178782299840241, 5.178782299840241, 5.178782299840241, 5.178495762786263, 4.626940002473793, 4.626940002473793, 4.626940002473793, 4.626940002473793, 4.626940002473793, 4.626940002473793, 4.626940002473793, 4.626940002473793, 4.626940002473793, 6.269185475817089, 6.269185475817089, 6.269185475817089, 6.269185475817089, 6.269185475817089, 6.269185475817089, 6.269185475817089, 6.269185475817089, 6.269185475817089, 6.269185475817089, 6.2689756708687545, 6.2689756708687545, 6.2689756708687545, 18.067065069850305, 18.067065069850305, 18.067065069850305, 18.067065069850305, 18.067065069850305, 18.066953617171432, 18.066953617171432, 18.066953617171432, 18.066953617171432, 18.066953617171432, 1.6546890202440376, 4.849291343893001, 4.849291343893001, 4.849291343893001, 4.849291343893001, 4.849291343893001, 4.849291343893001, 4.849291343893001, 4.849291343893001, 4.849291343893001, 4.849291343893001, 4.849291343893001, 4.849291343893001, 13.449192228814768, 18.170471501182632, 18.170471501182632, 18.170471501182632, 18.170471501182632, 18.170471501182632, 18.170471501182632, 17.335502351732476, 17.335502351732476, 17.335502351732476]

In [22]:
f3 = [3992.3794343011778, True, True, True, 4175.08264323352, True, True, True, True, True, 3981.1475283670097, 3981.1475283670097, True, True, True, True, True, True, True, 4093.915729185928, True, True, True, True, True, True, True, True, True, 4093.915729185928, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 4163.42537054753, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 3969.082524838492, 3969.082524838492, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 13.109191668224653, 3980.7867693838675, 13.109191668224653, 13.109191668224653, 2.180563611962972, 3980.7867693838675, 6.28819128044343, 6.28819128044343, 6.28819128044343, 6.28819128044343, 6.28819128044343, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 4165.816375281608, True, True, True, True, True, 52.105306582975274, 52.105306582975274, 52.105306582975274, True, 15.229613506068727, True, 4090.545249078355, True, True, True, True, 14.653576036972359, 4087.5621089015044, True, True, True, True, True, 4848.396465869255, 4848.396465869255, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 4061.3954478247542, True, True, True, True, True, True, True, True, True, True, True, 3967.0904859712646, True, True, True, True, True, True, True, True, True, True, True, 4110.141609437554, True, True, True, True, True, True, True, True, True, True, 3970.9429658031236, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 7.383852748651226, 6.7355638748728195, 16.61837659198573, 16.61837659198573, 2.298247564508458, True, True, True, True, True, True, True, True, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, True, True, True, True, True, True, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 7.383852748651226, 6.7355638748728195, 16.61837659198573, 16.61837659198573, 2.298247564508458, True, True, True, True, True, True, True, True, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.29400731317183, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, True, True, True, True, True, True, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596, 57.365255524903596]

In [24]:
f1.count(True)

33

In [17]:
f2.count(True)

106

In [18]:
f3.count(True)

341